In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.2.19,io.delta:delta-core_2.12:2.1.0 pyspark-shell'

from pyspark import StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
  .builder \
  .appName("Spark SQL") \
  .master("local") \
  .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
  .config("spark.sql.warehouse.dir", "../spark-warehouse") \
  .enableHiveSupport() \
  .getOrCreate()

In [2]:
cars_df = spark \
  .read \
  .json("data/cars") \
  .persist(StorageLevel.MEMORY_ONLY)

cars_df.show(10, 30) # показать 10 строк, обрубать длинные строки по 30 символу

In [3]:
# Spark DSL
american_cars_df = cars_df \
  .filter(col("Origin") == "Japan") \
  .select(col("Name"))

In [4]:
american_cars_df.explain()

In [5]:
american_cars_df.show(10, False) # показать 10 строк, не обрубать длинные строки

In [6]:
# Сохранить как таблицу в Spark, но не сохранять данные на диск
#  DataFrame => SQL metastore
print("DataFrame => SQL metastore = EXTERNAL TABLE")
cars_df.createOrReplaceTempView("cars")
spark.catalog.listTables()

In [7]:
# Spark SQL != Spark DSL
# Выполнить SQL запросы к DF, которые известны Apache Spark под какими-то именами
american_cars_df_v2 = spark.sql("SELECT Name FROM cars WHERE Origin = 'Japan'")
american_cars_df_v2.show(10, False)

In [8]:
# Временные таблицы действуют только на время жизни сессии
try:
  spark.newSession().sql("SELECT Name FROM cars WHERE Origin = 'Japan'")
except Exception as e:
  print (e)

In [9]:
# Удаление внешней таблицы (External Table) не удаляет данные на диске, только в metastore
spark.sql("DROP TABLE cars")

In [10]:
print("Ожидаемо падает:")
try:
  spark.sql("SELECT Name FROM cars WHERE Origin = 'Japan'")
except Exception as e:
  print(e)

In [11]:
# Но данные по прежнему доступны для чтения с диска
cars_df_again = spark.read.json("data/cars")
cars_df_again.count()

Сохранить датафрейм как Spark таблицу: `DataFrame => SQL metastore + Spark storage`

In [12]:
!find ~ -type d -name cars_managed_table | wc -l

In [13]:
cars_df \
  .write \
  .mode("overwrite") \
  .saveAsTable("cars_managed_table")

In [14]:
!find ~ -type d -name cars_managed_table -exec find {} \;

`saveAsTable` выполняет действия отличные от `save()` + `orc(...)` или `parquet(....)`.

В случае, например, `parquet()` указывается место для хранения файлов на диске:
```python
df.write \
  .parquet("data/parquet"). \
  save()
```

In [15]:
print("Чтение из управляемой таблицы (Managed Table) при помощи SQL запроса:")
american_cars_df_v2 = spark.sql("SELECT * FROM cars_managed_table")
american_cars_df_v2.show(10, False)

In [16]:
assert(cars_df.count() == american_cars_df_v2.count())

In [17]:
# spark.table == spark.read.table
# cars_managed_df = spark.read.table("cars_managed_table")

print("Чтение из управляемой таблицы (Managed Table) при помощи Spark DSL:")

cars_managed_df = spark.table("cars_managed_table")
assert (cars_managed_df.count() != 0)
cars_managed_df.show(10, False)

In [18]:
print("Удалить управляемую таблицу (Managed Table)")
spark.sql("DROP TABLE cars_managed_table")

In [19]:
!find ~ -type d -name cars_managed_table | wc -l

In [20]:
# Через Spark SQL можно создавать таблицы и вставлять записи
spark.sql("CREATE SCHEMA test")
spark.sql("CREATE TABLE test.students (name VARCHAR(64), address VARCHAR(64)) USING PARQUET PARTITIONED BY (student_id INT)")
spark.sql("""
INSERT INTO test.students
VALUES ('Bob Brown', '456 Taylor St, Cupertino', 222222)
     , ('Cathy Johnson', '789 Race Ave, Palo Alto', 333333)
""")

In [21]:
ddl_demo_df = spark.sql("SELECT * FROM test.students")
ddl_demo_df.show(10, False)

За дополнительными сведениями об INSERT можно обратиться к [документации](https://spark.apache.org/docs/latest/sql-ref-syntax-dml-insert-table.html)

In [22]:
print("Удалить схему test")
spark.sql("DROP SCHEMA test CASCADE")

## Delta Lake

[Delta](https://delta.io/learn/getting-started/) - современный формат эффективного хранения частоменяющихся данных

In [23]:
# сохранить первую версию
cars_df \
  .limit(5) \
  .write \
  .format("delta") \
  .mode("overwrite") \
  .save("../out/my_delta_cars")

In [24]:
# Прочитать текущую версию
delta_df = spark.read \
  .format("delta") \
  .load("../out/my_delta_cars")

In [25]:
delta_df.createOrReplaceTempView("my_delta_cars")

In [26]:
spark.sql("select * from my_delta_cars").show()

In [27]:
# дописать одну строку / записать новую версию
cars_df.limit(1) \
    .write \
    .format("delta") \
    .mode("append") \
    .save("../out/my_delta_cars")

In [28]:
spark.sql("select count(*) from my_delta_cars").show()

In [29]:
df = spark \
  .read \
  .format("delta") \
  .option("versionAsOf", 0) \
  .load("../out/my_delta_cars")

df.count()

# Задания

1. Получить список всех сотрудников и их максимальные зарплаты
1. Получить список всех сотрудников, кто никогда не был менеджером
1. Для каждого сотрудника, найти разницу между их зарплатой (текущей/последней) и максимальной зарплатой в их отделе

In [30]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://postgres:5432/spark"
user = "docker"
password = "docker"

In [31]:
def read_table(table_name):
    return spark.read. \
        format("jdbc"). \
        option("driver", driver). \
        option("url", url). \
        option("user", user). \
        option("password", password). \
        option("dbtable", "public." + table_name). \
        load()

employees_df = read_table("employees")
salaries_df = read_table("salaries")
dept_managers_df = read_table("dept_manager")
dept_emp_df = read_table("dept_emp")
departments_df = read_table("departments")

In [32]:
# save table names
employees_df.createOrReplaceTempView("employees")
salaries_df.createOrReplaceTempView("salaries")
dept_managers_df.createOrReplaceTempView("dept_manager")
dept_emp_df.createOrReplaceTempView("dept_emp")
departments_df.createOrReplaceTempView("departments")